In [61]:
import pandas as pd
import seaborn as sns
import numpy as np

In [62]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic']

In [63]:
df = sns.load_dataset('titanic')
df_main = df.copy()
df_main.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [64]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [65]:
df_main.describe().T

,count,mean,std,min,25%,50%,75%,max
survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
sibsp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [66]:
df_main.isnull().any()

survived       False
pclass         False
sex            False
age             True
sibsp          False
parch          False
fare           False
embarked        True
class          False
who            False
adult_male     False
deck            True
embark_town     True
alive          False
alone          False
dtype: bool

# -----------------------------------------------------------------

# Eksik Değerler Problemini Çözelim.


### Sayısal değişkenlerdeki eksik değerleri random forests fonksiyonu ile tahmine dayalı olarak dolduralım.

In [67]:
df_main.head(2)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False


In [68]:
df = df_main.select_dtypes(include=['float64', 'int64'])
df.head(2)

,survived,pclass,age,sibsp,parch,fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833


In [69]:
df.isnull().sum()

survived      0
pclass        0
age         177
sibsp         0
parch         0
fare          0
dtype: int64

In [70]:
df['age'].mean()

29.69911764705882

In [71]:
# veri setindeki değişkenlerin isimlerini bir listeye aktaralım.

var_names = list(df)
var_names

['survived', 'pclass', 'age', 'sibsp', 'parch', 'fare']

In [72]:
# veri setini ndarray formatına dönüştürelim. Random Forests fonksiyonu bizden bir ndarray bekliyor.

n_df = np.array(df)
n_df

array([[ 0.    ,  3.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  1.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 1.    ,  3.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 0.    ,  3.    ,     nan,  1.    ,  2.    , 23.45  ],
       [ 1.    ,  1.    , 26.    ,  0.    ,  0.    , 30.    ],
       [ 0.    ,  3.    , 32.    ,  0.    ,  0.    ,  7.75  ]])

In [73]:
# random forests algoritması ile age değişkenindeki eksik değerleri dolduralım.

from sklearn.ensemble import RandomForestRegressor

In [74]:
var_names

['survived', 'pclass', 'age', 'sibsp', 'parch', 'fare']

In [75]:
# eksik değerleri ve eksik olmayan değerleri ayrıştıralım.

dolu = df[pd.isnull(df['age']) == False] # eksik olmayan değerler
bos = df[pd.isnull(df['age'])] # eksik değerler

In [76]:
var_names.remove('age')

In [77]:
# modelimizi oluşturalım ve verileri modelimize verelim.

model = RandomForestRegressor()
model.fit(dolu[var_names], dolu['age'])     # modeli veriler ile eğittik.
new_ages = model.predict(X=bos[var_names])  # boş değerleri tahmin etmesini istedik.

bos['age'] = new_ages.astype(int) # tahmin edilen değerleri integer formatında veri setine ekledik.

df = dolu.append(bos) # eksik değerler ve eksik olmayan değerleri birleştirdik.

df.isnull().sum()

C:\Users\murat\AppData\Local\Temp\ipykernel_22392\3447862731.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bos['age'] = new_ages.astype(int) # tahmin edilen değerleri integer formatında veri setine ekledik.
C:\Users\murat\AppData\Local\Temp\ipykernel_22392\3447862731.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = dolu.append(bos) # eksik değerler ve eksik olmayan değerleri birleştirdik.


survived    0
pclass      0
age         0
sibsp       0
parch       0
fare        0
dtype: int64

In [81]:
# boş değerler varken age değişkeninin ortalaması 29.699, tahmin işleminden sonraki ortalama ise 29.562.

df_main['age'].mean(), df['age'].mean()

(29.69911764705882, 29.562480359147024)

In [87]:
# ana veri setine tahmin değerlerini ekleyelim.

df_main['age'][:] = df['age']
df_main.isnull().sum()

C:\Users\murat\AppData\Local\Temp\ipykernel_22392\2847665229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main['age'][:] = df['age']


survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

### --------------------------------------------------------------------------------

In [94]:
df_main.deck.isnull().sum()
# deck isimli değişkende 688 adet eksik veri var.

688

In [97]:
df_main.deck
# Bu değişkende 7 adet sınıf var. Sınıflar sıralı değil. Bu sınıflar ordinal mı nominal mı bilmiyorum.
# Değişken ismi güverte olduğu için bir sıralama olduğunu düşünmüyorum. Bu yüzden nominalden ordinale dönüştürme işlemi yapmayacağım.

0      NaN
1        C
2      NaN
3        C
4      NaN
      ... 
886    NaN
887      B
888    NaN
889      C
890    NaN
Name: deck, Length: 891, dtype: category
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']

#### Bu değişkendeki eksik değerleri modlarına bakarak yapacağım.

In [99]:
df_main['deck'].mode()

0    C
Name: deck, dtype: category
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [104]:
mode = df_main['deck'].mode()[0]
mode

'C'

In [107]:
df_main['deck'].fillna(mode, inplace=True)

In [108]:
df_main.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       2
class          0
who            0
adult_male     0
deck           0
embark_town    2
alive          0
alone          0
dtype: int64

In [115]:
mode = df_main['embarked'].mode()[0]
df_main['embarked'].fillna(mode, inplace=True)

In [114]:
mode = df_main['embark_town'].mode()[0]
df_main['embark_town'].fillna(mode, inplace=True)

In [116]:
df_main.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64

# --------------------------------------------------------